In [ ]:
!pip install python-dotenv
!pip install google-generativeai --upgrade
!pip install openai
!pip install elevenlabs
!pip install pydub
!pip install webrtcvad


In [ ]:
import os
from dotenv import load_dotenv

import openai
import elevenlabs
from elevenlabs import play
from google.colab import userdata
# import google.generativeai as genai
import google.generativeai as genai

import io
import time
import tempfile
from google.colab import output
from base64 import b64decode
import numpy as np
import io
from IPython.display import Audio, display, Javascript
import soundfile as sf
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
# First cell in Colab notebook
from google.colab import output
from IPython.display import Javascript, HTML, display

In [ ]:
from IPython.display import Javascript, Audio
from google.colab import output
import webrtcvad
import numpy as np
from base64 import b64decode, b64encode
import io

# Modified recording function with VAD
def record_audio(seconds_max=30, vad_mode=3, silence_threshold=1.5):
    # JavaScript for recording
    js_code = """
    // JavaScript code for recording with silence detection
    const sleep = time => new Promise(resolve => setTimeout(resolve, time))
    const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onloadend = e => resolve(e.srcElement.result)
        reader.readAsDataURL(blob)
    })

    async function record_with_vad(time_max) {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true })
        recorder = new MediaRecorder(stream)
        chunks = []

        recorder.ondataavailable = e => chunks.push(e.data)
        recorder.start(100) // Collect data in small chunks for analysis

        const startTime = Date.now()
        let silenceStart = null
        let silenceThreshold = 1500 // 1.5 seconds in milliseconds

        // Create audio context for analysis
        const audioContext = new AudioContext()
        const analyser = audioContext.createAnalyser()
        const microphone = audioContext.createMediaStreamSource(stream)
        microphone.connect(analyser)
        analyser.fftSize = 512
        const bufferLength = analyser.frequencyBinCount
        const dataArray = new Uint8Array(bufferLength)

        while ((Date.now() - startTime) < time_max * 1000) {
            // Analyze audio
            analyser.getByteFrequencyData(dataArray)
            let sum = dataArray.reduce((a, b) => a + b, 0)
            let average = sum / bufferLength

            if (average < 10) { // Low audio level threshold
                if (!silenceStart) {
                    silenceStart = Date.now()
                } else if ((Date.now() - silenceStart) > silenceThreshold) {
                    console.log("Silence detected, stopping")
                    break
                }
            } else {
                silenceStart = null
            }

            await sleep(100)
        }

        recorder.stop()
        await sleep(100) // Give time for ondataavailable to fire

        blob = new Blob(chunks)
        text = await b2text(blob)
        return text
    }
    """

    print("Recording... (speak naturally, recording will stop after silence)")
    display(Javascript(js_code))
    audio_data_b64 = output.eval_js(f'record_with_vad({seconds_max})')
    print("Recording finished.")

    # Decode the base64 audio data
    audio_binary = b64decode(audio_data_b64.split(',')[1])

    # Return both binary data and Audio widget
    return audio_binary#, Audio(audio_binary)




In [ ]:

# Initialize Gemini
#client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))

#model = genai.GenerativeModel('gemini-pro')
#model = genai('models/gemini-pro')

# Function to generate and play speech
def speak(text):
    # Instantiate ElevenLabs with your API key
    elevenlabs_instance = elevenlabs.ElevenLabs(api_key=userdata.get('ELEVENLABS_API_KEY'))

    # Generate and play audio using the instance
    audio_stream = elevenlabs_instance.generate(
        text=text,
        voice="Antoni",
        model="eleven_monolingual_v1"
    )

    play(audio_stream, elevenlabs_instance)

    return


def speak(text):
    # Instantiate ElevenLabs with your API key
    elevenlabs_instance = elevenlabs.ElevenLabs(api_key=userdata.get('ELEVENLABS_API_KEY'))

    # Generate and play audio using the instance
    audio_stream = elevenlabs_instance.generate(
        text=text,
        voice="Antoni",
        model="eleven_monolingual_v1",
        stream=True
    )

    audio_data = io.BytesIO()
    for chunk in audio_stream:
        audio_data.write(chunk)

    audio_data.seek(0)
    play(audio_data.getvalue(),elevenlabs_instance)

    # Calculate duration of audio (assuming 24kHz sample rate and 16-bit audio)
    duration = len(audio_data.getvalue()) / (24000)  # in seconds

    return duration


import io
import tempfile
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def transcribe(audio_data):
    # Create a temporary file to store the audio data
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as temp_audio_file:
        # Write the audio data to the temporary file
        temp_audio_file.write(audio_data)
        temp_audio_file.flush()

        # Rewind the file pointer to the beginning
        temp_audio_file.seek(0)

        # Perform transcription using the OpenAI API
        # Pass the file path to the 'file' argument instead of the file object
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=open(temp_audio_file.name, "rb") # Open the file in binary read mode
        )

    return transcript.text

'''
# Function to get AI response
def get_ai_response(prompt):
    # Use client to generate content instead of the undefined 'model' variable.
    # Using gemini-2.0-flash model, but consider using other available models.
    response = client.models.generate_content(
        model="gemini-2.0-flash-lite",  # Replace with desired model
        contents=prompt
    )
    return response.text
'''
# Configure the API
genai.configure(api_key=userdata.get('GEMINI_API_KEY'))

def get_ai_response(prompt):
    model = genai.GenerativeModel('gemini-2.0-flash-lite')
    response = model.generate_content(prompt)
    return response.text

In [ ]:
# Main conversation loop

context = "Elfi-homes. We offer regular system check that includes visual inspection of components, thermal imaging and cleaning as required. We recommend to use this making sure system is performing to its capabilities. 2 Checks/Year is recommended. We also offer installing EV charging station, Main Panel Upgrades and installing electric appliances. For the time being, we are offering first 4 inspections for free. The package cost is $1000 worth.   The package includes free cleaning and free system health check. It is important to keep an eye on the batteries and also making sure your solar system is producing proper electricity."

# Your AI response generation here
#ai_response = get_ai_response(f"")
#print("AI:", ai_response)
speak("Hi! Good evening. I am SalesGenie, a virtual Salesman. Am I speaking with Rohan and is this a good time to speak?")
# Wait until speech is finished
time.sleep(6)
audio = record_audio()
user_input = transcribe(audio)
ai_response = get_ai_response(f"You are a sales assistant. You are calling to talk about {context}. The person has responded he is available to talk. Get the person onboard with the idea")
speak(ai_response)

while True:
    # Record audio
    audio = record_audio()

    user_input = transcribe(audio)

    print("Transcribed text:", user_input)

    ai_response = get_ai_response(f"You are a sales assistant. You are calling to talk about {context}. Respond to: {user_input}")
    print("AI:", ai_response)
    speak(ai_response)

    if "goodbye" in user_input.lower():
        speak("Thank you for your time. Goodbye!")
        break

ApiError: status_code: 401, body: {'detail': {'status': 'quota_exceeded', 'message': 'This request exceeds your quota of 10000. You have 35 credits remaining, while 113 credits are required for this request.'}}

In [ ]:
""

''

In [ ]:
# Main conversation loop
print("Hello! I'm a virtual sales assistant. Are you available to talk?")

while True:
    # Here you would record audio, but for simplicity, we'll use text input
    user_input = input("Your response (text for now): ")

    # In a real implementation, you'd use:
    #audio_file = record_audio()
    #user_input = transcribe(audio_file)

    response = client.models.generate_content(
      model="gemini-2.0-flash", contents=f"You are a sales assistant. Respond to: {user_input}")
    print(response.text)

    #speak(ai_response)

    if "goodbye" in user_input.lower():
        speak("Thank you for your time. Goodbye!")
        break




Hello! I'm a virtual sales assistant. Are you available to talk?


KeyboardInterrupt: Interrupted by user